In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import LSTM
from keras.layers import GRU
from keras.layers import SimpleRNN
from keras.layers import Embedding
from keras.optimizers import RMSprop

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

In [ ]:
 # load
in_filename = '/content/drive/MyDrive/data/spam_seq_sel_done_ag3.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

In [ ]:
len(lines)

27447

In [ ]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1

In [ ]:
vocab_size

3214

In [ ]:
# separate into input and output
sequences = array(sequences)
X, y_real = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y_real, num_classes=vocab_size)

seq_length = X.shape[1]

In [ ]:
#!wget -P /content/drive/MyDrive/RNNs/pretrain "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2021-01-16 03:51:18--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.8.150
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.8.150|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘/content/drive/MyDrive/RNNs/pretrain/GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  73.3MB/s    in 23s     

2021-01-16 03:51:41 (68.1 MB/s) - ‘/content/drive/MyDrive/RNNs/pretrain/GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [ ]:
from sklearn.model_selection import StratifiedKFold
import numpy

In [ ]:
seed = 7
numpy.random.seed(seed)

In [ ]:
# define 10-fold cross validation test harness
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
cvscores = []

In [ ]:
#-P /content/drive/ 

In [ ]:
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from gensim.models.keyedvectors import KeyedVectors

In [ ]:
word_vectors = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/RNNs/pretrain/GoogleNews-vectors-negative300.bin.gz', binary=True)

In [ ]:
import numpy as np
EMBEDDING_DIM=300
NUM_WORDS=20000

embedding_matrix = np.zeros((vocab_size, EMBEDDING_DIM))

for word, i in tokenizer.word_index.items():
    if i>=NUM_WORDS:
        continue
    try:
        embedding_vector = word_vectors[word]
        embedding_matrix[i] = embedding_vector
    except KeyError:
        embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),EMBEDDING_DIM)

del(word_vectors)

In [ ]:
cvscores=[]
historymdl=[]
sc=[]
for train, test in kfold.split(X, y_real):
  # create model
  model=Sequential()
  model.add(Embedding(vocab_size,
                              EMBEDDING_DIM,
                              weights=[embedding_matrix],
                              trainable=True))
  #vocab_size, 50, input_lenth=seq_length
  #model.add(Dropout(0.2))
  #model.add(Conv1D(64, 5, activation='relu'))
  #model.add(MaxPooling1D(pool_size=4))
  model.add(LSTM(100))
  model.add(Dense(100, activation='relu'))
  model.add(Dense(vocab_size, activation='softmax'))
  print(model.summary())
  # Compile model
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  # Fit the model
  historym=model.fit(X[train], y[train], validation_data=(X[test], y[test]), batch_size=128, epochs=200)
  historymdl.append(historym) 
  # evaluate the model
  scores = model.evaluate(X[test], y[test], verbose=0)
  sc.append(scores)
  print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
  cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (numpy.mean(cvscores), numpy.std(cvscores)))

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 300)         964200    
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               160400    
_________________________________________________________________
dense_4 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_5 (Dense)              (None, 3214)              324614    
Total params: 1,459,314
Trainable params: 1,459,314
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/200
193/193 [==============================] - 6s 26ms/step - loss: 7.0038 - accuracy: 0.0415 - val_loss: 6.3046 - val_accuracy: 0.0561
Epoch 2/200
193/193 [==============================] - 5s 24ms/step - loss: 6.1068 - accuracy

In [ ]:
# save the model to file
model.save('/content/drive/MyDrive/RNNs/model/w2v_lstm_100.h5')

In [ ]:
#from keras.models import load_model
#model = load_model('/content/drive/MyDrive/RNNs/model/w2v_lstm_100.h5')

# continue training
#history200=model.fit(X_train, y_train, validation_split=0.33, batch_size=128, epochs=200)

In [ ]:
# save the tokenizer
dump(tokenizer, open('/content/drive/MyDrive/RNNs/tokenizer/w2v_lstm_100.pkl', 'wb'))

In [ ]:
import numpy as np
print("Average val_acc: ", np.average(historym.history['val_accuracy']))

import matplotlib.pyplot as plt
# summarize history for accuracy
plt.plot(historym.history['accuracy'])
plt.plot(historym.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(historym.history['loss'])
plt.plot(historym.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
cvscores

In [ ]:
model.metrics_names

In [ ]:
scores

In [ ]:
historym.history